# 🧩 FlexTrack Challenge — Modeling Pipeline Setup

**Notebook:** 02_modeling_setup.ipynb  
**Goal:** Build a reliable Phase 1 classification pipeline with time-aware, site-aware cross-validation and feature-rich LightGBM modeling.  

---

## 📌 Objectives
- Implement **cross-validation strategies** that prevent leakage:
  - Time-series folds (per-site, chronological)
  - Leave-one-site-out (LOSO) for robustness checks
- Validate split integrity (no future data leakage, no site overlap)
- Train and evaluate a **LightGBM classifier** with imbalance handling
- Establish the foundation for:
  - Threshold tuning
  - Feature engineering iterations
  - Phase 2 regression (capacity prediction)

---

## 🔑 Context Recap
- **Phase 1 Target:** `Demand_Response_Flag` (0 = no event, ±1 = event)  
- **Phase 2 Target:** `Demand_Response_Capacity_kW` (continuous, only when event ≠ 0)  
- **Key EDA insights:**  
  - Extreme class imbalance (~97% zeros)  
  - Events mostly during business hours, hot/sunny conditions  
  - Building power highly autocorrelated (lags/rolling stats are predictive)  
- **Feature blueprint:**  
  - Time-based (hour, weekday, month, business hours, weekend)
  - Weather (temperature, radiation, interactions)
  - Load dynamics (lags, rolling means/std, deltas, z-scores)
  - Site identity (one-hot encoding)
  - Interaction terms (e.g. temp × radiation)

## ⚙️ Setup & Imports
Minimal, reproducible environment for Phase 1 modeling:
- Core: NumPy, Pandas, Matplotlib
- Model: LightGBM (sklearn API)
- CV & Metrics: TimeSeriesSplit, confusion_matrix
- Quiet warnings for clean logs

In [1]:
# === Core ===
import os
import numpy as np
import pandas as pd
#import matplotlib.pyplot as plt
import warnings
from datetime import datetime

# === Modeling ===
import lightgbm as lgb
from lightgbm import LGBMClassifier

# === CV & Metrics ===
from sklearn.model_selection import TimeSeriesSplit
from sklearn.metrics import confusion_matrix

# === Misc ===
warnings.filterwarnings("ignore")

# Reproducibility
SEED = 42
np.random.seed(SEED)

print("Environment ready. Seed set to", SEED)

Environment ready. Seed set to 42


## 📂 Load Training & Test Data
We load both the processed **training data** and the official **public test set**.  
- `Timestamp_Local` is parsed as datetime for time-based validation and feature engineering.  
- Shapes are logged for confirmation.  

In [2]:
# --- Load train & test data ---
train_path = "../data/raw/flextrack-training-data-v0.1.csv"
test_path  = "../data/raw/flextrack-public-test-data-v0.1.csv"

df_train = pd.read_csv(train_path, parse_dates=["Timestamp_Local"])
df_test  = pd.read_csv(test_path,  parse_dates=["Timestamp_Local"])

print("Train shape:", df_train.shape)
print("Test shape:", df_test.shape)

Train shape: (105120, 7)
Test shape: (35040, 5)


## ⏳ Cross-Validation Strategy

We define a **time-aware, site-aware cross-validation splitter**.  
- Splits are created **independently within each site**, preserving temporal order.  
- Then, the site-level splits are combined to form global folds.  
- This ensures no leakage across time or sites.  

In [3]:
def make_site_timeseries_folds(df, n_splits=3):
    """
    Create time-aware, site-aware CV folds.

    Each site is split independently using TimeSeriesSplit,
    then site-level splits are combined into global folds.

    Parameters
    ----------
    df : pd.DataFrame
        Input dataframe sorted by ['Site', 'Timestamp_Local'].
    n_splits : int, default=3
        Number of folds.

    Returns
    -------
    folds : list of tuple
        List of (train_idx, val_idx) index arrays.
    """
    folds_per_site = {}
    for site, dsite in df.groupby("Site", sort=False):
        idx = dsite.index.to_numpy()
        tscv = TimeSeriesSplit(n_splits=n_splits)
        site_folds = [(idx[tr_rel], idx[va_rel]) for tr_rel, va_rel in tscv.split(idx)]
        folds_per_site[site] = site_folds

    folds = []
    for k in range(n_splits):
        tr_parts, va_parts = [], []
        for site in folds_per_site:
            tr_idx, va_idx = folds_per_site[site][k]
            tr_parts.append(tr_idx)
            va_parts.append(va_idx)
        folds.append((np.concatenate(tr_parts), np.concatenate(va_parts)))

    return folds

## 🌍 Leave-One-Site-Out (LOSO) Folds

As a **robust generalization check**, we build folds where:  
- Training happens on two sites  
- Validation happens on the **held-out third site**  

This mimics the scenario of applying the model to an unseen site.  

In [4]:
def make_leave_one_site_out_folds(df):
    """
    Create leave-one-site-out (LOSO) folds.

    For each site, train on all other sites and validate on the held-out site.

    Parameters
    ----------
    df : pd.DataFrame
        Input dataframe with a 'Site' column.

    Returns
    -------
    folds : list of tuple
        List of (train_idx, val_idx) index arrays.
    """
    folds = []
    all_idx = df.index.to_numpy()

    for site, dsite in df.groupby("Site", sort=False):
        val_idx = dsite.index.to_numpy()
        train_mask = np.isin(all_idx, val_idx, invert=True)
        tr_idx = all_idx[train_mask]
        folds.append((tr_idx, val_idx))

    return folds

## 🔎 Fold Integrity Check

To avoid **temporal leakage**, we define a helper that:  
- Summarizes the row counts and time ranges for train/valid splits  
- Verifies that, within each site, the **training period ends before validation begins**  

In [5]:
def summarize_fold(df, tr_idx, va_idx, label=""):
    """
    Summarize and validate the integrity of a single train/validation split.

    Parameters
    ----------
    df : pd.DataFrame
        Input dataframe containing 'Site' and 'Timestamp_Local'.
    tr_idx : array-like
        Row indices for the training set.
    va_idx : array-like
        Row indices for the validation set.
    label : str, optional
        Name/label for this fold (for printing).

    Raises
    ------
    AssertionError
        If any site's training period overlaps with its validation period.
    """
    tr, va = df.loc[tr_idx], df.loc[va_idx]

    print(f"{label} | Train: {tr.shape[0]:>6} rows  [{tr['Timestamp_Local'].min()} → {tr['Timestamp_Local'].max()}]")
    print(f"{' '*len(label)} | Valid: {va.shape[0]:>6} rows  [{va['Timestamp_Local'].min()} → {va['Timestamp_Local'].max()}]")

    # 🚨 Check for leakage per site
    for site in df['Site'].unique():
        tr_s, va_s = tr[tr['Site'] == site], va[va['Site'] == site]
        if not tr_s.empty and not va_s.empty:
            assert tr_s['Timestamp_Local'].max() < va_s['Timestamp_Local'].min(), f"Temporal leakage detected in site {site}"

## 📂 Cross-Validation Setup

To ensure robust evaluation, we prepare two complementary CV strategies:

- **TimeSeriesSplit (per-site):**  
  Splits each site into time-ordered chunks to prevent temporal leakage.  
  These are combined across sites to form 3 primary folds.

- **Leave-One-Site-Out (LOSO):**  
  Trains on two sites and validates on the third.  
  This serves as a stricter robustness check for site generalization.

Both folds are validated with integrity checks to confirm no overlap or leakage.

In [6]:
# --- Build CV folds (primary + robustness) ---
ts_folds   = make_site_timeseries_folds(df_train, n_splits=3)   # primary CV
loso_folds = make_leave_one_site_out_folds(df_train) 

## ✅ Cross-Validation Integrity Check

The folds were successfully built and verified:

- **TimeSeriesSplit (3 folds):**  
  - Each fold uses past data for training and later periods for validation.  
  - Train/valid boundaries respect chronological order and prevent leakage.  
  - Each validation set has ~26k rows (≈ 25% of site data).  

- **Leave-One-Site-Out (LOSO, 3 folds):**  
  - Each fold trains on two sites and validates on the held-out site.  
  - Ensures robustness to site-level distribution shifts.  
  - Each validation set has 35k rows (one full site).  

All checks confirmed no overlap or leakage between training and validation ranges.

In [7]:
# --- Integrity summaries ---
for i, (tr_idx, va_idx) in enumerate(ts_folds, 1):
    summarize_fold(df_train, tr_idx, va_idx, label=f"TS Fold {i}")
for i, (tr_idx, va_idx) in enumerate(loso_folds, 1):
    summarize_fold(df_train, tr_idx, va_idx, label=f"LOSO Fold {i}")

TS Fold 1 | Train:  26280 rows  [2019-01-01 00:00:00 → 2023-04-02 05:45:00]
          | Valid:  26280 rows  [2019-04-02 06:00:00 → 2023-07-02 11:45:00]
TS Fold 2 | Train:  52560 rows  [2019-01-01 00:00:00 → 2023-07-02 11:45:00]
          | Valid:  26280 rows  [2019-07-02 12:00:00 → 2023-10-01 17:45:00]
TS Fold 3 | Train:  78840 rows  [2019-01-01 00:00:00 → 2023-10-01 17:45:00]
          | Valid:  26280 rows  [2019-10-01 18:00:00 → 2023-12-31 23:45:00]
LOSO Fold 1 | Train:  70080 rows  [2019-01-01 00:00:00 → 2023-12-31 23:45:00]
            | Valid:  35040 rows  [2019-01-01 00:00:00 → 2019-12-31 23:45:00]
LOSO Fold 2 | Train:  70080 rows  [2019-01-01 00:00:00 → 2023-12-31 23:45:00]
            | Valid:  35040 rows  [2019-01-01 00:00:00 → 2019-12-31 23:45:00]
LOSO Fold 3 | Train:  70080 rows  [2019-01-01 00:00:00 → 2019-12-31 23:45:00]
            | Valid:  35040 rows  [2023-01-01 00:00:00 → 2023-12-31 23:45:00]


## 📊 G-Mean from Hard Predictions

To handle **class imbalance**, we compute the geometric mean (G-Mean) of sensitivity and specificity.  
This helper:  
- Derives TPR (recall for events) and TNR (specificity for non-events)  
- Returns G-Mean = √(TPR · TNR) along with both components  
- Works directly on **hard 0/1 predictions**  

In [8]:
def gmean_binary(y_true, y_pred):
    """
    Compute G-Mean, TPR, and TNR from binary predictions.

    Parameters
    ----------
    y_true : array-like of shape (n_samples,)
        Ground-truth labels (0 = non-event, 1 = event).
    y_pred : array-like of shape (n_samples,)
        Hard predictions (0/1).

    Returns
    -------
    gmean : float
        Geometric mean of TPR and TNR.
    tpr : float
        True Positive Rate (recall for the positive class).
    tnr : float
        True Negative Rate (specificity for the negative class).
    """
    tn, fp, fn, tp = confusion_matrix(y_true, y_pred, labels=[0, 1]).ravel()
    tpr = tp / (tp + fn) if (tp + fn) else 0.0
    tnr = tn / (tn + fp) if (tn + fp) else 0.0
    return float((tpr * tnr) ** 0.5), float(tpr), float(tnr)

## 📊 G-Mean from Probability Thresholds

To better handle class imbalance, we sweep across probability thresholds:  
- Convert probabilities into hard 0/1 predictions at each threshold  
- Compute **G-Mean, TPR, and TNR**  
- Select the threshold that maximizes G-Mean  

This avoids relying on the default `0.5` cutoff, which is often poor for imbalanced data.

In [9]:
def gmean_from_probs(y_true, y_prob, thresholds):
    """
    Sweep thresholds to maximize G-Mean.

    Parameters
    ----------
    y_true : array-like of shape (n_samples,)
        Ground-truth labels (0 = non-event, 1 = event).
    y_prob : array-like of shape (n_samples,)
        Predicted probabilities for the positive class.
    thresholds : array-like
        Candidate thresholds to evaluate.

    Returns
    -------
    best_gm : float
        Best geometric mean of TPR and TNR across thresholds.
    best_thr : float
        Threshold that achieved the best G-Mean.
    best_tpr : float
        True Positive Rate (recall for the positive class) at best threshold.
    best_tnr : float
        True Negative Rate (specificity for the negative class) at best threshold.
    """
    best = (-1.0, 0.5, 0.0, 0.0)  # (gm, thr, tpr, tnr)

    for thr in thresholds:
        y_pred = (y_prob >= thr).astype(int)
        tn, fp, fn, tp = confusion_matrix(y_true, y_pred, labels=[0,1]).ravel()

        tpr = tp / (tp + fn) if (tp+fn) > 0 else 0.0
        tnr = tn / (tn + fp) if (tn+fp) > 0 else 0.0
        gm  = np.sqrt(tpr * tnr)

        if gm > best[0]:
            best = (gm, thr, tpr, tnr)

    return best

## 🧱 Feature Engineering

Build leakage-safe features capturing **time**, **weather**, and **load dynamics**:

- **Time**: `hour`, `dow`, `month`, `bizhrs` (10–17), `wknd` (Sat/Sun).
- **Weather**: `temp` (Dry_Bulb_Temperature_C), `rad` (Global_Horizontal_Radiation_W/m2), `temp_x_rad` (interaction).
- **Load lags (per-site)**: `power_lag1`, `power_lag4`, `power_lag12`, `power_lag24`, `power_lag96`.
- **Rolling stats (per-site)**: `power_roll4`, `power_roll12`, `power_roll96`, `power_std96`, `power_min96`, `power_max96`.
- **Daily context**: `power_dev96` (current − 24h mean), `power_rel96` (current ÷ 24h mean).
- **Change & scale**: `power_delta` (diff vs lag1), `power_z` (per-site z-score).
- **Site identity**: one-hot `Site_*`.
- **Target**: `y_event = 1` if `Demand_Response_Flag != 0` else `0` (if available; `NaN` for test).

_All rolling/lag features are computed within each site and only use past data to avoid temporal leakage._

In [10]:
def build_features(df: pd.DataFrame) -> pd.DataFrame:
    """
    Construct model features from raw FlexTrack columns.

    Parameters
    ----------
    df : pd.DataFrame
        Must contain at least:
        - 'Site', 'Timestamp_Local', 'Building_Power_kW',
        - 'Dry_Bulb_Temperature_C', 'Global_Horizontal_Radiation_W/m2'
        Optionally: 'Demand_Response_Flag' (if present, y_event is derived).

    Returns
    -------
    pd.DataFrame
        Copy of input with engineered features and one-hot 'Site_*' columns.
        If 'Demand_Response_Flag' exists, includes binary target 'y_event'.
    """
    d = df.copy()

    # --- datetime ordering ---
    d = d.sort_values(["Site", "Timestamp_Local"])

    # --- target (if available) ---
    if "Demand_Response_Flag" in d.columns:
        d["y_event"] = (d["Demand_Response_Flag"] != 0).astype(int)
    else:
        d["y_event"] = np.nan

    # --- time features ---
    d["hour"]   = d["Timestamp_Local"].dt.hour
    d["dow"]    = d["Timestamp_Local"].dt.dayofweek
    d["month"]  = d["Timestamp_Local"].dt.month
    d["bizhrs"] = d["hour"].between(10, 17).astype(int)
    d["wknd"]   = d["dow"].isin([5, 6]).astype(int)

    # --- weather features ---
    d["temp"] = d["Dry_Bulb_Temperature_C"]
    d["rad"]  = d["Global_Horizontal_Radiation_W/m2"]
    d["temp_x_rad"] = d["temp"] * d["rad"]

    # --- load lags (per-site) ---
    for lag in [1, 4, 12, 24, 96]:
        d[f"power_lag{lag}"] = d.groupby("Site", sort=False)["Building_Power_kW"].shift(lag)

    # --- rolling stats (per-site) ---
    grp = d.groupby("Site", sort=False)["Building_Power_kW"]
    d["power_roll4"]   = grp.transform(lambda s: s.rolling(4,  min_periods=1).mean())
    d["power_roll12"]  = grp.transform(lambda s: s.rolling(12, min_periods=1).mean())
    d["power_roll96"]  = grp.transform(lambda s: s.rolling(96, min_periods=1).mean())
    d["power_std96"]   = grp.transform(lambda s: s.rolling(96, min_periods=10).std())
    d["power_min96"]   = grp.transform(lambda s: s.rolling(96, min_periods=10).min())
    d["power_max96"]   = grp.transform(lambda s: s.rolling(96, min_periods=10).max())

    # --- deviations / ratios / deltas ---
    eps = 1e-6
    d["power_dev96"] = d["Building_Power_kW"] - d["power_roll96"]
    d["power_rel96"] = d["Building_Power_kW"] / (d["power_roll96"] + eps)
    d["power_delta"] = d["Building_Power_kW"] - d["power_lag1"]

    # --- per-site z-score of power ---
    d["power_z"] = d.groupby("Site", sort=False)["Building_Power_kW"].transform(
        lambda s: (s - s.mean()) / (s.std() + 1e-6)
    )

    # --- one-hot site ---
    d = pd.get_dummies(d, columns=["Site"], drop_first=False)

    return d

## 🧱 Build Features (Train & Test)

To ensure consistency across both splits, we apply the unified `build_features()` pipeline to train and test data.  

This step:  
- Generates **time, weather, and load-derived features**  
- Adds **site one-hot encodings**  
- Creates the `y_event` target for training data (left as `NaN` for test)  
- Guarantees no leakage by only using **current or past values** (lags/rollings)  

In [11]:
# --- Build engineered features for train & test ---
df_feat_train = build_features(df_train)
df_feat_test  = build_features(df_test)

## 🔄 Align One-Hot Encoded Site Columns

After one-hot encoding `Site`, the **train and test sets may not have identical dummy columns**:  
- If a site is missing in the test set, its dummy column won’t be created.  
- If an unexpected site appears in the test set, it won’t exist in the training set.  

To prevent feature mismatches at inference time, we:  
- Collect the list of all `Site_*` columns in both train and test 

In [12]:
# --- Collect site dummy columns in train & test ---
train_sites = [c for c in df_feat_train.columns if c.startswith("Site_")]
test_sites  = [c for c in df_feat_test.columns  if c.startswith("Site_")]

## 🧩 Ensure Feature Consistency Between Train and Test

To guarantee both datasets have the **same set of `Site_*` dummy columns**:  
- Add any missing site columns in the test set and fill them with `0`  
- Drop any unexpected site columns in the test set (safety check)  

This ensures the **train and test feature matrices are aligned** before modeling.  

In [13]:
# --- Ensure feature consistency between train & test ---
missing_sites = set(train_sites) - set(test_sites)
extra_sites   = set(test_sites) - set(train_sites)

# Add missing site dummies to test (fill with 0)
for col in missing_sites:
    df_feat_test[col] = 0

# Drop extra site dummies from test (safety check)
if extra_sites:
    df_feat_test.drop(columns=list(extra_sites), inplace=True)

## 🧮 Define Final Feature Set

We explicitly select the features created in `build_features`:  
- **Time features** → `hour`, `dow`, `month`, `bizhrs`, `wknd`  
- **Weather features** → `temp`, `rad`, `temp_x_rad`  
- **Load lags** → `power_lag1`, `power_lag4`, `power_lag12`, `power_lag24`, `power_lag96`  
- **Rolling statistics** → `power_roll4`, `power_roll12`, `power_roll96`, `power_std96`  
- **Daily context** → `power_min96`, `power_max96`, `power_dev96`, `power_rel96`  
- **Dynamics** → `power_delta`, `power_z`  
- **Site identity** → all `Site_*` dummy variables  

Finally, we restrict the list to columns that exist in **both train and test** so that the feature matrices remain aligned.

In [14]:
# --- Define feature list to match `build_features` ---
feature_cols = [
    # time
    "hour", "dow", "month", "bizhrs", "wknd",

    # weather
    "temp", "rad", "temp_x_rad",

    # load lags
    "power_lag1", "power_lag4", "power_lag12", "power_lag24", "power_lag96",

    # rolling stats (24h window = 96 steps)
    "power_roll4", "power_roll12", "power_roll96", "power_std96",
    "power_min96", "power_max96",

    # deviations / ratios / deltas / normalization
    "power_dev96", "power_rel96", "power_delta", "power_z",
] + [c for c in df_feat_train.columns if c.startswith("Site_")]

# --- Ensure only features present in BOTH train & test ---
feature_cols_final = [
    c for c in feature_cols
    if c in df_feat_train.columns and c in df_feat_test.columns
]

print("Num features:", len(feature_cols_final))

Num features: 26


## 🧪 Time-Aware CV with Threshold Tuning

We evaluate models using **time- and site-aware folds**, combined with a **threshold sweep** to handle class imbalance:

- Train LightGBM on each fold with early stopping  
- Sweep probability thresholds to maximize **G-Mean**  
- Record the best threshold and per-fold metrics  

This provides a **leakage-safe validation framework** and a principled way to choose cutoffs instead of relying on the default `0.5`.

In [15]:
def run_cv_threshold_tuned(df_feat, folds, feature_cols, thresholds=None):
    """
    Run time-aware CV with LightGBM and per-fold threshold tuning.

    Parameters
    ----------
    df_feat : pd.DataFrame
        Feature matrix including target `y_event`.
    folds : list[tuple]
        List of (train_idx, val_idx) index splits.
    feature_cols : list[str]
        Columns used for training.
    thresholds : array-like, optional
        Candidate probability thresholds. Default: np.linspace(0.02, 0.5, 25).

    Returns
    -------
    scores : list[float]
        Best G-Mean per fold.
    chosen_thrs : list[float]
        Best threshold per fold.
    per_fold_stats : list[tuple]
        (gmean, thr, tpr, tnr, pos, neg, pos_ratio) per fold.
    """
    if thresholds is None:
        thresholds = np.linspace(0.02, 0.5, 25)

    scores, chosen_thrs, per_fold_stats = [], [], []

    for i, (tr_idx, va_idx) in enumerate(folds, 1):
        tr = df_feat.loc[tr_idx].dropna(subset=feature_cols + ["y_event"])
        va = df_feat.loc[va_idx].dropna(subset=feature_cols + ["y_event"])

        X_tr, y_tr = tr[feature_cols], tr["y_event"]
        X_va, y_va = va[feature_cols], va["y_event"]

        # Imbalance handling: sqrt(neg/pos)
        pos = max((y_tr == 1).sum(), 1)
        neg = (y_tr == 0).sum()
        spw = (neg / pos) ** 0.5

        clf = LGBMClassifier(
            random_state=42,
            n_estimators=1500,
            learning_rate=0.03,
            num_leaves=127,
            max_depth=-1,
            min_data_in_leaf=20,
            subsample=0.8,
            colsample_bytree=0.8,
            reg_lambda=1.0,
            scale_pos_weight=float(spw),
            n_jobs=-1,
            verbosity=-1
        )

        callbacks = [
            lgb.early_stopping(stopping_rounds=150, verbose=False),
            lgb.log_evaluation(period=0)  # silence eval logging
        ]

        clf.fit(
            X_tr, y_tr,
            eval_set=[(X_va, y_va)],
            eval_metric="binary_logloss",
            callbacks=callbacks
        )

        # Probabilities (use best iteration if available)
        try:
            y_prob = clf.predict_proba(X_va, num_iteration=clf.best_iteration_)[:, 1]
        except Exception:
            y_prob = clf.predict_proba(X_va)[:, 1]

        # Threshold sweep
        best_gm, best_thr, best_tpr, best_tnr = gmean_from_probs(y_va, y_prob, thresholds)

        pos_val = int((y_va == 1).sum())
        neg_val = int((y_va == 0).sum())
        pos_ratio = pos_val / max(len(y_va), 1)

        scores.append(best_gm)
        chosen_thrs.append(best_thr)
        per_fold_stats.append((best_gm, best_thr, best_tpr, best_tnr, pos_val, neg_val, pos_ratio))

        bi = getattr(clf, "best_iteration_", None)
        print(
            f"Fold {i}  | GM={best_gm:.3f} | Thr={best_thr:.3f} | "
            f"TPR={best_tpr:.3f} | TNR={best_tnr:.3f} | "
            f"Pos={pos_val}, Neg={neg_val}, PosRatio={pos_ratio:.3%} "
            + (f"| BestIter={bi}" if bi is not None else "")
        )

    avg_pos_ratio = float(np.mean([s[6] for s in per_fold_stats])) if per_fold_stats else 0.0
    print(f"\nCV G-Mean: {np.mean(scores):.3f} ± {np.std(scores):.3f}")
    print(f"Avg validation positive ratio: {avg_pos_ratio:.3%}")

    thr_list = [float(t) for t in chosen_thrs]   # <- fixed (was thrs_tt)
    print("Thresholds:", [f"{t:.3f}" for t in thr_list])

    return scores, chosen_thrs, per_fold_stats

## ▶️ Run Threshold-Tuned CV

Using the **time-series folds**, we now:  
- Train LightGBM with early stopping  
- Tune thresholds per fold to maximize **G-Mean**  
- Collect cross-validation scores, thresholds, and stats  

This gives us a reliable **local validation estimate** before submitting predictions.

In [16]:
# --- Run threshold-tuned cross-validation (time-series folds) ---
scores_tt, thrs_tt, stats_tt = run_cv_threshold_tuned(
    df_feat_train, ts_folds, feature_cols
)

Fold 1  | GM=0.487 | Thr=0.020 | TPR=0.256 | TNR=0.928 | Pos=481, Neg=25799, PosRatio=1.830% | BestIter=30
Fold 2  | GM=0.792 | Thr=0.020 | TPR=0.911 | TNR=0.689 | Pos=895, Neg=25385, PosRatio=3.406% | BestIter=13
Fold 3  | GM=0.347 | Thr=0.040 | TPR=0.123 | TNR=0.979 | Pos=756, Neg=25524, PosRatio=2.877% | BestIter=13

CV G-Mean: 0.542 ± 0.186
Avg validation positive ratio: 2.704%
Thresholds: ['0.020', '0.020', '0.040']


## 📈 Cross-Validation Results (Time-Series Folds)

After running **threshold-tuned cross-validation** with LightGBM, we obtain:  

- Per-fold performance metrics (G-Mean, threshold, TPR, TNR)  
- Validation class distribution (positive/negative counts and ratio)  
- Best boosting iteration selected via early stopping  

This provides a clear picture of how well the model generalizes across different time segments.  

**Summary:**  
- Average CV G-Mean: **0.542 ± 0.186**  
- Average validation positive ratio: **2.704%**  
- Chosen thresholds per fold: `['0.020', '0.020', '0.040']`  

## ▶️ Run Threshold-Tuned CV (LOSO)

Using **leave-one-site-out (LOSO)** folds, we:  
- Train on two sites and validate on the **held-out site**  
- Tune per-fold thresholds to maximize **G-Mean**  
- Assess **cross-site generalization** (how well the model transfers to unseen sites)  

In [17]:
# --- Run threshold-tuned cross-validation (leave-one-site-out folds) ---
scores_loso, thrs_loso, stats_loso = run_cv_threshold_tuned(
    df_feat_train, loso_folds, feature_cols
)

Fold 1  | GM=0.815 | Thr=0.020 | TPR=0.739 | TNR=0.898 | Pos=771, Neg=34173, PosRatio=2.206% | BestIter=55
Fold 2  | GM=0.850 | Thr=0.020 | TPR=0.816 | TNR=0.885 | Pos=1109, Neg=33835, PosRatio=3.174% | BestIter=52
Fold 3  | GM=0.049 | Thr=0.040 | TPR=0.002 | TNR=1.000 | Pos=1229, Neg=33715, PosRatio=3.517% | BestIter=2

CV G-Mean: 0.571 ± 0.369
Avg validation positive ratio: 2.966%
Thresholds: ['0.020', '0.020', '0.040']


## 🌍 Cross-Validation Results (LOSO Folds)

After running **threshold-tuned cross-validation** with LightGBM, we obtain:  

- Per-fold performance metrics (G-Mean, threshold, TPR, TNR)  
- Validation class distribution (positive/negative counts and ratio)  
- Best boosting iteration selected via early stopping  

This provides a strict test of **site-to-site generalization**, where the model is trained on two sites and validated on the held-out third site.  

**Summary:**  
- Average CV G-Mean: **0.571 ± 0.369**  
- Average validation positive ratio: **2.966%**  
- Chosen thresholds per fold: `['0.020', '0.020', '0.040']`  

## 🏋️‍♂️ Prepare Final Training Matrix

Before fitting the full model, we:  
- Drop any rows with **NaNs** in features or target (`y_event`)  
- Extract the finalized **feature matrix (`X`)** and **target vector (`y`)**  
- Print the dataset dimensions to confirm the training setup  

This ensures the model is trained on a **clean, consistent dataset** aligned with our CV pipeline.  

In [18]:
# --- Prepare training matrix (clean, NaN-free) ---
df_train_nn = df_feat_train.dropna(subset=feature_cols_final + ["y_event"]).copy()

X = df_train_nn[feature_cols_final]
y = df_train_nn["y_event"].astype(int)
print(f"Training rows: {len(X):,} | Features: {X.shape[1]}")

Training rows: 104,832 | Features: 26


## ▶️ Train Final Model & Generate Submission

We now train a **final LightGBM** model on the full training data using the **same settings as cross-validation** to ensure consistency.  

- **Class imbalance handling**: `scale_pos_weight = sqrt(neg/pos)`  
- **Threshold**: mean of CV-tuned thresholds (fallback = `0.02`)  
- **Submission format**:  
  - `-1` → event predicted  
  - `0` → no event  

This produces the final submission file aligned with the competition requirements.  

In [19]:
# --- Train final model with CV-consistent settings ---
# Class imbalance weight (same recipe as CV): sqrt(neg/pos)
pos = max(int((y == 1).sum()), 1)
neg = int((y == 0).sum())
spw = (neg / pos) ** 0.5

clf = LGBMClassifier(
    random_state=42,
    n_estimators=1500,
    learning_rate=0.03,
    num_leaves=127,
    max_depth=-1,
    min_data_in_leaf=20,
    subsample=0.8,
    colsample_bytree=0.8,
    reg_lambda=1.0,
    scale_pos_weight=float(spw),
    n_jobs=-1,
    verbosity=-1
)

clf.fit(X, y)

# --- Predict on test ---
X_test = df_feat_test[feature_cols_final]
y_prob = clf.predict_proba(X_test)[:, 1]

# --- Use mean threshold from CV if available; else default to 0.02 ---
FIXED_THR = float(np.mean([float(t) for t in thrs_loso])) if "thrs_loso" in locals() else 0.02
y_pred = (y_prob >= FIXED_THR).astype(int)

# --- Map to competition labels: event -> -1, no event -> 0 ---
y_flag = np.where(y_pred == 1, -1, 0)

# --- Build submission from ORIGINAL test df (preserve Site & Timestamp) ---
sub = pd.DataFrame({
    "Site": df_test["Site"].values,
    "Timestamp_Local": pd.to_datetime(df_test["Timestamp_Local"]).dt.strftime("%Y-%m-%d %H:%M:%S"),
    "Demand_Response_Flag": y_flag
})

# --- Quick sanity prints ---
print(f"Predicted events in test: {(sub['Demand_Response_Flag'] != 0).sum()} / {len(sub)}")
print(f"Using threshold: {FIXED_THR:.3f} | pos={pos}, neg={neg}, scale_pos_weight={spw:.3f}")

Predicted events in test: 1326 / 35040
Using threshold: 0.027 | pos=3109, neg=101723, scale_pos_weight=5.720


## ✅ Final Model Training & Test Predictions

The full model was trained with **CV-consistent settings** (same parameters and imbalance handling as in cross-validation).  

**Key outcomes:**  
- **Predicted events in test:** 1,326 / 35,040 rows (≈ **3.8%**)  
- **Threshold applied:** 0.027 (mean from CV, more conservative than baseline 0.020)  
- **Training distribution:** 3,109 positives vs. 101,723 negatives  
- **Class weight (scale_pos_weight):** 5.72  

This ensures the **final submission** is aligned with cross-validation insights, balancing event detection against false alarms.

In [20]:
# --- Save submission with informative filename ---
today = datetime.now().strftime("%Y-%m-%d")  # current date
model_tag = "lgbm"                           # model identifier
cv_tag = "loso-cv"                             # cross-validation strategy
version = "v1"                               # manual version tag

# Construct filename and path
filename = f"submission_{today}_{model_tag}_{cv_tag}_{version}.csv"
save_path = f"../submissions/{filename}"

# Ensure folder exists and save CSV
#os.makedirs(os.path.dirname(save_path), exist_ok=True)
sub.to_csv(save_path, index=False)

print(f"Submission saved: {save_path} | shape={sub.shape}")

Submission saved: ../submissions/submission_2025-09-05_lgbm_loso-cv_v1.csv | shape=(35040, 3)


In [22]:
# --- Leak-free threshold tuning using LOSO folds (retrain per fold) ---

candidate_thrs = np.linspace(0.001, 0.030, 15)  # tighter, low range
best_thr, best_cv_gm = None, -1.0

for thr in candidate_thrs:
    fold_gms = []
    for tr_idx, va_idx in loso_folds:
        # Per-fold train/valid split + dropna (exactly like CV)
        tr = df_feat_train.loc[tr_idx].dropna(subset=feature_cols_final + ["y_event"])
        va = df_feat_train.loc[va_idx].dropna(subset=feature_cols_final + ["y_event"])

        X_tr, y_tr = tr[feature_cols_final], tr["y_event"]
        X_va, y_va = va[feature_cols_final], va["y_event"]

        # Same imbalance weighting as CV: sqrt(neg/pos)
        pos = max((y_tr == 1).sum(), 1)
        neg = (y_tr == 0).sum()
        spw = (neg / pos) ** 0.5

        # Same model params as CV
        clf_cv = LGBMClassifier(
            random_state=42,
            n_estimators=1500,
            learning_rate=0.03,
            num_leaves=127,
            max_depth=-1,
            min_data_in_leaf=20,
            subsample=0.8,
            colsample_bytree=0.8,
            reg_lambda=1.0,
            scale_pos_weight=float(spw),
            n_jobs=-1,
            verbosity=-1
        )

        callbacks = [
            lgb.early_stopping(stopping_rounds=150, verbose=False),
            lgb.log_evaluation(period=0),
        ]
        clf_cv.fit(
            X_tr, y_tr,
            eval_set=[(X_va, y_va)],
            eval_metric="binary_logloss",
            callbacks=callbacks
        )

        # Predict VA and compute G-Mean at the *single* candidate threshold
        y_prob_va = clf_cv.predict_proba(X_va, num_iteration=clf_cv.best_iteration_)[:, 1]
        gm, _, _, _ = gmean_from_probs(y_va, y_prob_va, [thr])
        fold_gms.append(gm)

    avg_gm = float(np.mean(fold_gms))
    print(f"thr={thr:.3f} | LOSO CV G-Mean={avg_gm:.3f}")
    if avg_gm > best_cv_gm:
        best_cv_gm, best_thr = avg_gm, thr

print(f"\nChosen threshold (LOSO): {best_thr:.3f} | CV G-Mean={best_cv_gm:.3f}")

# --- Apply the chosen threshold to your already-trained full model ---
y_pred = (y_prob >= best_thr).astype(int)          # y_prob came from the full-model on test
y_flag = np.where(y_pred == 1, -1, 0)
sub["Demand_Response_Flag"] = y_flag

# Save tuned submission
from datetime import datetime
today = datetime.now().strftime("%Y-%m-%d")
filename = f"submission_{today}_lgbm_loso-cv_thr{best_thr:.3f}.csv"
save_path = f"../submissions/{filename}"
os.makedirs(os.path.dirname(save_path), exist_ok=True)
sub.to_csv(save_path, index=False)

print(f"Submission saved: {save_path} | events={int((sub['Demand_Response_Flag']!=0).sum())}/{len(sub)}")

thr=0.001 | LOSO CV G-Mean=0.000
thr=0.003 | LOSO CV G-Mean=0.000
thr=0.005 | LOSO CV G-Mean=0.000
thr=0.007 | LOSO CV G-Mean=0.567
thr=0.009 | LOSO CV G-Mean=0.578
thr=0.011 | LOSO CV G-Mean=0.569
thr=0.013 | LOSO CV G-Mean=0.565
thr=0.015 | LOSO CV G-Mean=0.562
thr=0.018 | LOSO CV G-Mean=0.559
thr=0.020 | LOSO CV G-Mean=0.556
thr=0.022 | LOSO CV G-Mean=0.552
thr=0.024 | LOSO CV G-Mean=0.546
thr=0.026 | LOSO CV G-Mean=0.670
thr=0.028 | LOSO CV G-Mean=0.663
thr=0.030 | LOSO CV G-Mean=0.658

Chosen threshold (LOSO): 0.026 | CV G-Mean=0.670
Submission saved: ../submissions/submission_2025-09-05_lgbm_loso-cv_thr0.026.csv | events=1336/35040
